In [27]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset

from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

data_dir = "../data/processed/"

In [28]:
# data loading 
train_file = np.loadtxt(data_dir+"windowed_train_FD001.txt",delimiter=",")
test_file = np.loadtxt(data_dir +"standardized_test_FD001.txt",delimiter=",")

train_x = train_file[:,2:]
train_y = train_file[:,1]

test_units = test_file[:,0]
test_x = test_file[:,2:]
test_y = test_file[:,1]


In [3]:
N_tw = 50
num_samples = int(train_file.shape[0]/N_tw)
num_features = 24

data_x = torch.zeros(num_samples,N_tw,num_features)
data_y = torch.zeros(num_samples,N_tw,1)

for i in range(num_samples):
    for t in range(N_tw):
        data_y[i][t][0] = torch.tensor(train_y[i*N_tw+t])
        data_x[i][t][:] = torch.tensor(train_x[i*N_tw+t,:])
        # for j in range(num_features):
        #     data_x[i][t][0][j] = train_x[i*N_tw+t,j]

In [4]:
class EngineDataset(Dataset):
    def __init__(self,data_x,data_y):
        self.x = data_x
        self.y = data_y

    def __len__(self):
        return self.x.size()[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [6]:
batch_size = 128
train_data = EngineDataset(data_x, data_y)
print(train_data.__len__())
print(train_data.x.size())
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
print(next(iter(train_loader))[0].size())

15731
torch.Size([15731, 50, 24])
torch.Size([128, 50, 24])


In [7]:
#set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [8]:
#Create Neural Net
class GRUNet(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,n_layers,drop_prob=0.5):
        super(GRUNet,self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #batch_first = True : input dims are (batch_size,seq_length,n_features)
        self.gru = nn.GRU(input_dim,hidden_dim,n_layers,batch_first=True,dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim,output_dim)
        self.relu = nn.ReLU()

    def forward(self,x,h):
        out,h = self.gru(x,h)
        out = self.fc(self.relu(out[:,-1]))

        return out,h

    def init_hidden(self,batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers,batch_size,self.hidden_dim).zero_().to(device)
        
        return hidden

In [21]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5):
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].size()[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)

    model.to(device)

    batch_size = train_loader.batch_size
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of GRU model")
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.process_time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            h = h.data

            model.zero_grad()
            label = label[:,-1]
            
            out, h = model(x.to(device).float(), h)

            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.process_time()        
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

In [24]:
lr = 0.001
gru_model = train(train_loader,lr,EPOCHS=50)

Starting Training of GRU model
Epoch 1/50 Done, Total Loss: 0.11341557288389714
Total Time Elapsed: 0.859375 seconds
Epoch 2/50 Done, Total Loss: 0.04943844676017761
Total Time Elapsed: 0.6875 seconds
Epoch 3/50 Done, Total Loss: 0.04173457969102214
Total Time Elapsed: 0.6875 seconds
Epoch 4/50 Done, Total Loss: 0.0394445481084165
Total Time Elapsed: 0.71875 seconds
Epoch 5/50 Done, Total Loss: 0.03924207396988497
Total Time Elapsed: 0.671875 seconds
Epoch 6/50 Done, Total Loss: 0.03590816042584474
Total Time Elapsed: 0.671875 seconds
Epoch 7/50 Done, Total Loss: 0.034963297428654845
Total Time Elapsed: 0.65625 seconds
Epoch 8/50 Done, Total Loss: 0.034588453207226075
Total Time Elapsed: 0.671875 seconds
Epoch 9/50 Done, Total Loss: 0.032883298690201804
Total Time Elapsed: 0.703125 seconds
Epoch 10/50 Done, Total Loss: 0.03158769362651911
Total Time Elapsed: 0.6875 seconds
Epoch 11/50 Done, Total Loss: 0.02972192892835277
Total Time Elapsed: 0.671875 seconds
Epoch 12/50 Done, Total Los

In [41]:
def evaluate(model, test_x, test_y, test_units):
    model.eval()
    outputs = []
    targets = []
    start_time = time.process_time()

    for i in range(1,100+1):
        ind = np.where(test_units == i)
        unit_x = test_x[ind,:]
        unit_y = test_y[ind]

        inp = torch.from_numpy(unit_x)
        labs = torch.from_numpy(unit_y)
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        out = 65*(out+1)
        target = 65*(unit_y[-1]+1)

        outputs.append(out)
        targets.append(target)

        
    print("Evaluation Time: {}".format(str(time.process_time()-start_time)))
    sMAPE = 0
    MAE = 0
    for i in range(len(outputs)):
        MAE += abs(outputs[i] - targets[i])/len(outputs)
        # sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print(f"MAE: {MAE.item()}")
    return
    return outputs, targets, sMAPE

evaluate(gru_model,test_x,test_y,test_units)

Evaluation Time: 0.421875
MAE: 11.453980445861816
